In [ ]:
!pip install pandas dash

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

spacex_df.head()

/var/folders/fq/yttnzmd967g5rwp9jhhh0zjr0000gn/T/ipykernel_1362/4073084833.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/fq/yttnzmd967g5rwp9jhhh0zjr0000gn/T/ipykernel_1362/4073084833.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options = [
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                            value = 'ALL',
                                            placeholder="Select Launch Site",
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=2500,
                                                value=[min_payload, max_payload],
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500:'7500', 10000:'10000'},
                                               ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    print(entered_site)
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df,
                    values='class',
                    names = 'Launch Site',
                    title = 'Launch Success for All Sites')
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site].groupby(by='class').count().reset_index()
        fig = px.pie(filtered_df,
                     values='Launch Site',
                     names='class',
                     title = f'Launch Success for {entered_site}')
        return fig
    
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), 
            Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df
    min_payload = payload_range[0]
    max_payload = payload_range[1]
    if entered_site =='ALL':    
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(min_payload, max_payload)]
        fig = px.scatter(data_frame = filtered_df, 
                         x='Payload Mass (kg)', 
                         y='class',
                        color='Booster Version')
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(min_payload, max_payload)]
        fig = px.scatter(data_frame = filtered_df, 
                         x='Payload Mass (kg)', 
                         y='class',
                        color='Booster Version')
        return fig
        

# Run the app
if __name__ == '__main__':
    app.run_server()



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2022 21:56:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


ALL


127.0.0.1 - - [10/Dec/2022 21:56:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:28] "POST /_dash-update-component HTTP/1.1" 200 -


CCAFS SLC-40


127.0.0.1 - - [10/Dec/2022 21:56:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:42] "POST /_dash-update-component HTTP/1.1" 200 -


CCAFS LC-40


127.0.0.1 - - [10/Dec/2022 21:56:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2022 21:56:51] "POST /_dash-update-component HTTP/1.1" 200 -
